In [1]:
import pandas as pd
import numpy as np

## Read in response variable:

In [2]:
response_variable = pd.read_csv('response_variable_v2.csv')
response_variable = response_variable.iloc[:,[0,2]]

## Matthew's features:

In [3]:
matthew_frame = pd.read_csv('features_matthew_v2.csv')

## Chris's features:

In [4]:
chris_frame_pc = pd.read_csv('pc_diagnosis.csv')

# Confirm that Matthew's model is running still:

In [5]:
df = response_variable.merge(matthew_frame, on='id')
df = df.set_index('id')

# Training/test split

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X = df.iloc[:,1:]
X = X[['SUPPLY_CNT_on_day0', 'PAYABLE_QTY_on_day0', 'total_SUPPLY_CNT_prior', 'total_PAYABLE_QTY_prior']]
y = df.iloc[:,0].map(lambda x: 1 if x == True else 0 )

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate

neigh = KNeighborsClassifier(n_neighbors=135)

cv_results = cross_validate(neigh, X_train, y_train, cv=10,
                            scoring= 'roc_auc',
                            return_train_score=True,
                           return_estimator =True)

np.mean(cv_results['test_score'])

0.8665545465491604

In [8]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, max_depth=7,
                                  random_state=100)

cv_results = cross_validate(rf_model, X_train, y_train, cv=10,
                            scoring= 'roc_auc',
                            return_train_score=True,
                           return_estimator =True)

np.mean(cv_results['test_score'])

0.8693006222087076

# Let's see if adding some principal components helps at all:

In [11]:
df = response_variable.merge(matthew_frame, on='id')
df = pd.merge(df, chris_frame_pc, on = 'id', how = 'left')
df = df.iloc[:,:-946]
df = df.set_index('id')

for col in df.select_dtypes(include=np.number):
    df[col] = df[col].fillna(df[col].median())
    
X = df.iloc[:,1:]
y = df.iloc[:,0].map(lambda x: 1 if x == True else 0 )

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)

rf_model = RandomForestClassifier(n_estimators=100, max_depth=10,
                                  random_state=100)

cv_results = cross_validate(rf_model, X_train, y_train, cv=3,
                            scoring= 'roc_auc',
                            return_train_score=True,
                           return_estimator =True)

np.mean(cv_results['test_score'])

0.8714024371081931

In [10]:
df

,LTOT_v2,MME_on_day0,SUPPLY_CNT_on_day0,PAYABLE_QTY_on_day0,max_MME_prior,avg_MME_prior,total_SUPPLY_CNT_prior,total_PAYABLE_QTY_prior,PC 1,PC 2,PC 3,PC 4
id,,,,,,,,,,,,
ID10010854159,False,15.000000,5.0,15.0,0.000000,0.000000,0.0,0.0,-0.008862,-0.001108,-0.009221,-0.006047
ID10013863216,True,10.000000,90.0,180.0,0.000000,0.000000,0.0,0.0,-0.009147,-0.001073,-0.009114,-0.005969
ID10024447278,True,50.000000,3.0,20.0,22.500000,20.000000,15.0,40.0,-0.009147,-0.001073,-0.009114,-0.005969
ID1002482139,True,60.000000,30.0,120.0,90.000000,37.254464,789.0,2895.0,-0.011015,0.008488,-0.002925,-0.006238
ID1003386406,False,20.000000,15.0,60.0,50.000000,50.000000,3.0,20.0,-0.009147,-0.001073,-0.009114,-0.005969
ID10036289882,False,37.500000,2.0,15.0,0.000000,0.000000,0.0,0.0,-0.009147,-0.001073,-0.009114,-0.005969
ID10036954381,False,37.500000,2.0,15.0,37.500000,37.500000,2.0,15.0,-0.009564,-0.001091,-0.011219,-0.007729
ID10055739763,True,15.000000,5.0,15.0,30.000000,30.000000,3.0,12.0,-0.008927,-0.001142,-0.009317,-0.006180
ID10074598346,False,20.000000,5.0,20.0,40.000000,25.842105,452.0,1480.0,-0.009202,-0.001098,-0.009828,-0.006526


Adding some principal components adds a tiny bit of accuracy, but not worth it at this point (probably within margin of error). I would like to try to retool and do better, but regradless it seems to be working well. 